# Common Information – Data Seeder (MongoDB Atlas)
Notebook ini menyiapkan **dataset FAQ** (pengiriman, pembelian, refund, dsb), membuat embedding, lalu menyimpannya ke **MongoDB Atlas** pada koleksi **`common_information`** sehingga bisa diretrieval oleh tool RAG.

In [5]:
# --- Setup & Imports
import os
import dotenv
from typing import List, Dict

from haystack.dataclasses import Document
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

dotenv.load_dotenv()
MONGO_CONNECTION_STRING = os.environ.get("MONGO_CONNECTION_STRING")
assert MONGO_CONNECTION_STRING, "Harap set MONGO_CONNECTION_STRING di .env"

DB_NAME = "depato_store"
COLLECTION = "common_information"
VECTOR_INDEX = "vector_index_common_info"
FTS_INDEX = "search_index_common_info"

In [6]:
seed_data: List[Dict] = [
    {
        "question": "Bagaimana cara pembelian di toko ini?",
        "answer": "Anda dapat membeli melalui website: pilih produk, tambah ke keranjang, isi alamat, lalu pilih metode pembayaran (transfer bank, e-wallet, kartu).",
        "category": "pembelian",
        "tags": ["cara pembelian", "order", "pembayaran"]
    },
    {
        "question": "Apa saja metode pembayaran yang tersedia?",
        "answer": "Kami menerima transfer bank (BCA/BNI/BRI/Mandiri), e-wallet (OVO/Gopay/DANA), dan kartu kredit/debit berlogo Visa/Mastercard.",
        "category": "pembelian",
        "tags": ["pembayaran", "metode pembayaran"]
    },
    {
        "question": "Berapa lama pengiriman barang?",
        "answer": "Pengiriman reguler umumnya 2–5 hari kerja, tergantung kurir dan lokasi. Tersedia opsi same day/next day di kota tertentu.",
        "category": "pengiriman",
        "tags": ["estimasi kirim", "durasi", "kurir"]
    },
    {
        "question": "Bagaimana cara melacak pesanan saya?",
        "answer": "Nomor resi dikirim via email/WhatsApp setelah pesanan diproses. Lacak melalui halaman ‘Order Tracking’ atau situs kurir.",
        "category": "pengiriman",
        "tags": ["resi", "tracking", "order tracking"]
    },
    {
        "question": "Bagaimana kebijakan refund dan pengembalian dana?",
        "answer": "Refund dapat diajukan maksimal 7 hari sejak barang diterima untuk kerusakan/pesanan tidak sesuai. Proses 3–5 hari kerja setelah disetujui.",
        "category": "refund",
        "tags": ["refund", "pengembalian dana", "retur"]
    },
    {
        "question": "Bagaimana cara retur/penukaran barang?",
        "answer": "Ajukan melalui halaman ‘Bantuan’ → ‘Retur/Penukaran’, sertakan foto/video bukti. Tim kami akan memvalidasi dan memberi label pengembalian.",
        "category": "refund",
        "tags": ["retur", "tukar barang", "klaim"]
    },
    {
        "question": "Apakah ada gratis ongkir?",
        "answer": "Gratis ongkir tersedia untuk pesanan di atas Rp300.000 atau saat promo tertentu. Syarat dapat berubah mengikuti kampanye berjalan.",
        "category": "pengiriman",
        "tags": ["ongkir", "gratis ongkir", "promo"]
    },
    {
        "question": "Bagaimana jika paket hilang atau rusak saat pengiriman?",
        "answer": "Segera hubungi CS dengan bukti foto/video. Kami bantu klaim ke kurir. Penggantian/kompensasi mengikuti hasil investigasi kurir.",
        "category": "pengiriman",
        "tags": ["klaim", "asuransi", "kerusakan"]
    }
]

In [7]:
document_store = MongoDBAtlasDocumentStore(
    database_name=DB_NAME,
    collection_name=COLLECTION,
    vector_search_index=VECTOR_INDEX,
    full_text_search_index=FTS_INDEX,
)

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [8]:
docs: List[Document] = []
for item in seed_data:
    content = f"""Q: {item['question']}
A: {item['answer']}"""
    meta = {"category": item["category"], "tags": item["tags"], "topic": "common_information"}
    docs.append(Document(content=content, meta=meta))

embedded = doc_embedder.run(documents=docs)
embedded_docs = embedded.get("documents", docs)

document_store.write_documents(embedded_docs)
print(f"Sukses menulis {len(embedded_docs)} dokumen ke koleksi '{COLLECTION}'.")

RuntimeError: The embedding model has not been loaded. Please call warm_up() before running.